# Encontro 13: Medidas de Centralidade



### Leonardo Medeiros

Importando a biblioteca:

In [1]:
import sys
sys.path.append('..')

from random import choice
from itertools import permutations

import pandas as pd
import networkx as nx

import socnet as sn

Configurando a biblioteca:

In [2]:
sn.node_size = 10
sn.node_color = (255, 255, 255)

sn.edge_width = 1
sn.edge_color = (192, 192, 192)

sn.node_label_position = 'top center'

Carregando rede de casamentos entre famílias de Florença durante a Renascença.

J. F. Padgett e C. K. Ansell. *Robust action and the rise of the Medici, 1400–1434.* American Journal of
Sociology 98, págs. 1259-1319, 1993.

In [3]:
g = sn.load_graph('Renaissance.gml', has_pos=True)

sn.show_graph(g, nlab=True)

Função que registra, em cada nó, seus sucessores em geodésicas de $s$ a $t$.

In [4]:
def set_geodesic_successors(g, s, t):
    for n in g.nodes:
        g.nodes[n]['geodesic_successors'] = set()

    for p in nx.all_shortest_paths(g, s, t):
        for i in range(len(p) - 1):
            g.nodes[p[i]]['geodesic_successors'].add(p[i + 1])

Funções que representam uma escolha aleatória de sucessor para diferentes tipos de trajetórias.

In [5]:
# Pense que o atributo 'passages' abaixo indica quantas
# vezes um fluxo já passou por um nó ou por uma aresta.

def random_geodesic_successor(g, n):
    return choice([m for m in g.nodes[n]['geodesic_successors']])

def random_path_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.nodes[m]['passages'] == 0])

def random_trail_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.edges[n, m]['passages'] == 0])

def random_walk_successor(g, n):
    return choice([m for m in g.neighbors(n)])

funcs = [random_geodesic_successor, random_path_successor, random_trail_successor, random_walk_successor ]

Função que faz uma simulação de fluxo de $s$ a $t$, que pode ou não ser bem-sucedida.

In [6]:
def simulate_single_flow(g, s, t, tipo, transferencia):
    # Inicializa o atributo 'passages' de cada nó.
    for n in g.nodes:
        g.nodes[n]['passages'] = 0
    g.nodes[s]['passages'] = 1

    # Inicializa o atributo 'passages' de cada aresta.
    for n, m in g.edges:
        g.edges[n, m]['passages'] = 0

    # Inicializa s como o único dono do insumo.
    for n in g.nodes:
        g.nodes[n]['owner'] = False
    g.nodes[s]['owner'] = True

    # Simula o fluxo, contando o número total de passos.

    steps = 0

    while True:
        # O conjunto reached representa todos os nós
        # que o fluxo consegue alcançar no passo atual.
        reached = set()

        # Verifica cada um dos donos atuais do insumo.

        owners = [n for n in g.nodes if g.nodes[n]['owner']]

        for n in owners:
            # Escolhe aleatoriamente um dos sucessores.
            try:
                m = funcs[tipo](g, n)
                
            except IndexError:
                continue

                
            if(transferencia):
                # Deixa de ser dono do insumo.
                g.nodes[n]['owner'] = False

            # Incrementa o atributo 'passages' do nó.
            g.nodes[m]['passages'] += 1

            # Incrementa o atributo 'passages' da aresta.
            g.edges[n, m]['passages'] += 1

            # Registra que consegue alcançar esse nó.
            reached.add(m)

        # Todo nó alcançado passa a ser dono do insumo.

        for n in reached:
            g.nodes[n]['owner'] = True

        # Isso conclui o passo atual da simulação.
        steps += 1

        # Se o passo alcançou t, chegamos ao fim da simulação.
        # Ela foi bem-sucedida: devolvemos o número de passos.
        if t in reached:
            return steps

        # Se o passo não alcançou ninguém, chegamos ao fim da
        # simulação. Ela não foi bem-sucedida: devolvemos -1.
        if not reached:
            return -1

Função que faz simulações de fluxo de $s$ a $t$ até uma ser bem-sucedida.

In [7]:
def simulate_successful_flow(g, s, t, tipo, transferencia):
    set_geodesic_successors(g, s, t)

    while True:
        steps = simulate_single_flow(g, s, t, tipo, transferencia)

        if steps != -1:
            return steps

Função que faz simulações de fluxo para todo $s$ e $t$ possíveis, e tira disso um *closeness simulado* e um *betweenness simulado*.

In [8]:
def simulate_all_flows(g, tipo, transferencia):
    for n in g.nodes:
        g.nodes[n]['closeness'] = 0
        g.nodes[n]['betweenness'] = 0

    for s, t in permutations(g.nodes, 2):
        steps = simulate_successful_flow(g, s, t, tipo, transferencia)

        g.nodes[s]['closeness'] += steps
        for n in g.nodes:
            if n != s and n != t:
                g.nodes[n]['betweenness'] += g.nodes[n]['passages']

    # Normalizações necessárias para comparar com os
    # resultados analíticos. Não é preciso entender.
    for n in g.nodes:
        g.nodes[n]['closeness'] = (g.number_of_nodes() - 1) / g.nodes[n]['closeness']
        g.nodes[n]['betweenness'] /= (g.number_of_nodes() - 1) * (g.number_of_nodes() - 2)

E agora, vamos pensar um pouco...

* Onde você precisa mudar o código para usar uma *trajetória* que não seja a *geodésica*? (caminho, trilha, passeio)

* Onde você precisa mudar o código para usar uma *difusão* que não seja a *transferência*? (duplicação)

Considere então a seguinte **hipótese**:

>Quando consideramos outros tipos de trajetória e outros tipos de difusão, os nós com maior *closeness simulado* e *betweenness simulado* não são necessariamente os nós com maior *closeness* e *betweenness* segundo as fórmulas clássicas. (que correspondem ao uso de geodésica e transferência na simulação)

Queremos:

1. Operacionalização e teste dessas hipótese. (Objetivo 3)
2. Interpretação dos resultados na linguagem de Análise de Redes Sociais (Objetivo 4)

Um *feedback* da atividade sobre *coreness no Jazz* será dado em breve, para vocês terem uma melhor referência do item 2.

In [9]:
def main(g, tipo, transferencia):
    TIMES = 100
    
    cc = nx.closeness_centrality(g)
    bc = nx.betweenness_centrality(g)

    for n in g.nodes:
        g.nodes[n]['mean_closeness'] = 0
        g.nodes[n]['mean_betweenness'] = 0

    for _ in range(TIMES):
        simulate_all_flows(g, tipo, transferencia)

        for n in g.nodes:
            g.nodes[n]['mean_closeness'] += g.nodes[n]['closeness']
            g.nodes[n]['mean_betweenness'] += g.nodes[n]['betweenness']

    for n in g.nodes:
        g.nodes[n]['mean_closeness'] /= TIMES
        g.nodes[n]['mean_betweenness'] /= TIMES
        
    return pd.DataFrame({
    'família': [g.nodes[n]['label'] for n in g.nodes],
    'closeness simulado': [g.nodes[n]['mean_closeness'] for n in g.nodes],
    'closeness analítico': [cc[n] for n in g.nodes],
    'betweenness simulado': [g.nodes[n]['mean_betweenness'] for n in g.nodes],
    'betweenness analítico': [bc[n] for n in g.nodes],
    })

## Dataframes

In [10]:
geodesic = [main(g, 0, True), main(g, 0, False)]
for i in geodesic:
    display(i)

,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
0,ginori,0.333333,0.333333,0.000000,0.000000
1,lambertes,0.325581,0.325581,0.000000,0.000000
2,albizzi,0.482759,0.482759,0.212033,0.212454
3,guadagni,0.466667,0.466667,0.256538,0.260073
4,pazzi,0.285714,0.285714,0.000000,0.000000
5,salviati,0.388889,0.388889,0.142857,0.142857
6,medici,0.560000,0.560000,0.522143,0.521978
7,tornabuon,0.482759,0.482759,0.088516,0.091575
8,bischeri,0.400000,0.400000,0.120000,0.120879
9,ridolfi,0.482759,0.482759,0.089560,0.086081


,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
0,ginori,0.333333,0.333333,0.000000,0.000000
1,lambertes,0.325581,0.325581,0.000000,0.000000
2,albizzi,0.482759,0.482759,0.573681,0.212454
3,guadagni,0.466667,0.466667,0.687912,0.260073
4,pazzi,0.285714,0.285714,0.000000,0.000000
5,salviati,0.388889,0.388889,0.406593,0.142857
6,medici,0.560000,0.560000,1.388681,0.521978
7,tornabuon,0.482759,0.482759,0.243791,0.091575
8,bischeri,0.400000,0.400000,0.314890,0.120879
9,ridolfi,0.482759,0.482759,0.249835,0.086081


In [11]:
path = [main(g, 1, True), main(g, 1, False)]
for i in path:
    display(i)

,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
0,ginori,0.205941,0.333333,0.000000,0.000000
1,lambertes,0.217135,0.325581,0.000000,0.000000
2,albizzi,0.252957,0.482759,0.229286,0.212454
3,guadagni,0.264643,0.466667,0.409121,0.260073
4,pazzi,0.199368,0.285714,0.000000,0.000000
5,salviati,0.241536,0.388889,0.142857,0.142857
6,medici,0.298622,0.560000,0.629176,0.521978
7,tornabuon,0.254980,0.482759,0.336484,0.091575
8,bischeri,0.243028,0.400000,0.375275,0.120879
9,ridolfi,0.254021,0.482759,0.374286,0.086081


,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
0,ginori,0.230938,0.333333,0.389231,0.000000
1,lambertes,0.236539,0.325581,0.396813,0.000000
2,albizzi,0.257127,0.482759,0.617363,0.212454
3,guadagni,0.263402,0.466667,0.739835,0.260073
4,pazzi,0.205930,0.285714,0.259670,0.000000
5,salviati,0.223707,0.388889,0.438846,0.142857
6,medici,0.264164,0.560000,0.819670,0.521978
7,tornabuon,0.278760,0.482759,0.675220,0.091575
8,bischeri,0.265928,0.400000,0.649560,0.120879
9,ridolfi,0.277411,0.482759,0.695055,0.086081


In [12]:
trail = [main(g, 2, True), main(g, 2, False)]
for i in trail:
    display(i)

,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
0,ginori,0.212874,0.333333,0.000000,0.000000
1,lambertes,0.204299,0.325581,0.000000,0.000000
2,albizzi,0.278852,0.482759,0.266758,0.212454
3,guadagni,0.267456,0.466667,0.408187,0.260073
4,pazzi,0.190588,0.285714,0.000000,0.000000
5,salviati,0.243650,0.388889,0.142857,0.142857
6,medici,0.300216,0.560000,0.705330,0.521978
7,tornabuon,0.250874,0.482759,0.317253,0.091575
8,bischeri,0.229338,0.400000,0.342967,0.120879
9,ridolfi,0.243103,0.482759,0.331264,0.086081


,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
0,ginori,0.222781,0.333333,0.370000,0.000000
1,lambertes,0.229796,0.325581,0.390934,0.000000
2,albizzi,0.247440,0.482759,0.685055,0.212454
3,guadagni,0.251687,0.466667,0.899670,0.260073
4,pazzi,0.198854,0.285714,0.267473,0.000000
5,salviati,0.218603,0.388889,0.428022,0.142857
6,medici,0.254398,0.560000,1.252692,0.521978
7,tornabuon,0.266436,0.482759,0.932582,0.091575
8,bischeri,0.250111,0.400000,0.785879,0.120879
9,ridolfi,0.261233,0.482759,0.928132,0.086081


In [13]:
walk = [main(g, 3, True), main(g, 3, False)]
for i in walk:
    display(i)

,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
0,ginori,0.038777,0.333333,0.732473,0.000000
1,lambertes,0.037770,0.325581,0.752143,0.000000
2,albizzi,0.037787,0.482759,2.404615,0.212454
3,guadagni,0.035242,0.466667,3.239066,0.260073
4,pazzi,0.039816,0.285714,0.699396,0.000000
5,salviati,0.038621,0.388889,1.527802,0.142857
6,medici,0.035516,0.560000,4.894890,0.521978
7,tornabuon,0.036821,0.482759,2.400110,0.091575
8,bischeri,0.036779,0.400000,2.421099,0.120879
9,ridolfi,0.034746,0.482759,2.403901,0.086081


,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
0,ginori,0.140033,0.333333,0.819451,0.000000
1,lambertes,0.143355,0.325581,0.702418,0.000000
2,albizzi,0.159052,0.482759,3.111264,0.212454
3,guadagni,0.157671,0.466667,4.565110,0.260073
4,pazzi,0.128303,0.285714,0.764121,0.000000
5,salviati,0.144738,0.388889,1.809780,0.142857
6,medici,0.167851,0.560000,6.984560,0.521978
7,tornabuon,0.167167,0.482759,2.430000,0.091575
8,bischeri,0.156206,0.400000,2.804066,0.120879
9,ridolfi,0.168699,0.482759,2.553736,0.086081


## Teste das Hipóteses